# Maximum Likelihood Estimation with Normal Distribution

## Import modules

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import sys
import time
import glob

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Setting hyperparameters

In [ ]:
# Training Flags (hyperparameter configuration)
max_epochs = 10
batch_size = 128
learning_rate = 1e-1

## Make a toy dataset (normal distribution)

In [ ]:
true_mu = -3.0
true_std = 3.5
N = 10000
train_data = np.random.normal(loc=true_mu, scale=true_std, size=N)
train_data = train_data.astype(np.float32)
train_data = np.expand_dims(train_data, axis=1)

## Set up dataset with `tf.data`

### create input pipeline with `tf.data.Dataset`

In [ ]:
# for train
N = len(train_data)
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
train_dataset = train_dataset.shuffle(buffer_size=N)
train_dataset = train_dataset.batch(batch_size=batch_size, drop_remainder=True)
print(train_dataset)

## Create the parameters to learn

**Normal distribution**

$$ \mathcal{N}(\mu, \sigma) = \frac{1}{\sqrt{2\pi \sigma^{2}}} \exp \left( {-\frac{(x-\mu)^{2}}{2\sigma^{2}}} \right) $$

**Log normal distribution**

$$ \log \mathcal{N}(\mu, \sigma) = -\frac{1}{2} \log(2 \pi \sigma^{2}) + \left[ -\frac{(x-\mu)^{2}}{2\sigma^{2}} \right] $$

$$ = -\frac{1}{2} \left[ \log(2 \pi) + \log(\sigma^{2}) + \frac{(x-\mu)^{2}}{\sigma^{2}} \right] $$

$$ = -\frac{1}{2} \left[ \log(2 \pi) + \log(\sigma^{2}) + (x-\mu)^{2} \exp(\log(-\sigma^{2})) \right] $$

**Variables**

* `mu`: $\mu$
* `logvar`: $\log (\sigma^{2})$

In [ ]:
# Intialize training values
mu = tf.Variable(0.0, name='mean')
logvar = tf.Variable(1.0, name='log_variance')

In [ ]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return -.5 * tf.reduce_sum((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi, axis=raxis)

## Define the loss functions and the optimizer

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate)

## Training

### Define training one step function

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(data, mu, logvar):
  with tf.GradientTape() as tape:
    negative_log_likelihood = -tf.reduce_mean(log_normal_pdf(data, mu, logvar))
      
    gradients = tape.gradient(negative_log_likelihood, [mu, logvar])
    optimizer.apply_gradients(zip(gradients, [mu, logvar]))
  
  return negative_log_likelihood

### Training full steps

In [ ]:
print('Start Training.')
num_batches_per_epoch = int(N / batch_size)
global_step = tf.Variable(0, trainable=False)

for epoch in range(max_epochs):
  
  for step, data in enumerate(train_dataset):

    negative_log_likelihood = train_step(data, mu, logvar)
    global_step.assign_add(1)

    if global_step.numpy() % 50 == 0:
      epochs = epoch + step / float(num_batches_per_epoch)
      #display.clear_output(wait=True)
      print("Epochs: {:.2f} global_step: {} loss: {:.3g}  mu: {:.3g}  std: {:3g}".format(
                epochs, global_step.numpy(), negative_log_likelihood, mu.numpy(), tf.sqrt(tf.exp(logvar)).numpy()))
      
print('Training Done.')

## Print the results

In [ ]:
print("Results")
print("mean: {:.3g}".format(mu.numpy()))
print("standard deviation: {:.3g}".format(np.sqrt(np.exp(logvar.numpy()))))
print("true mean: {:.3g}".format(true_mu))
print("true standard deviation: {:.3g}".format(true_std))